In [1]:
%load_ext watermark
%watermark

2020-09-16T10:55:51-05:00

CPython 3.7.6
IPython 7.13.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.4.0-47-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [2]:
import pandas as pd

noticias = pd.read_csv("noticias.csv")
noticias.head()

,descripcion,categoria
0,"Aunque parezca mentira, las emisiones de dióxi...",cultura
1,Hubo un proyecto impulsado por la Unión Europe...,cultura
2,China ha confirmado la conclusión con éxito de...,tecnología
3,"En su fructífera carrera como humorista, actor...",cultura
4,Tras dos años de negociación entre la instituc...,cultura


El objetivo sera predecir la categoria de una noticia dada la descripción de esta

In [3]:
noticias.categoria.value_counts()

cultura       9001
tecnología    4198
ocio          3296
Name: categoria, dtype: int64

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

Vamos a modificar este vectorizador para modificar dos parametros **strip_accents** que quita los acentos y **stop_words** que elimina las palabras que no tienen nigún sentido semantico es decir que no aportan nada al significado de la frase

### Eliminar Stopwords

In [5]:
import json

with open("stopwords-es.json") as fname:
    stopwords_es = json.load(fname)

vectorizador = TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es)

In [6]:
stopwords_es[:30]

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '_',
 'a',
 'actualmente',
 'acuerdo',
 'adelante',
 'ademas',
 'además',
 'adrede',
 'afirmó',
 'agregó',
 'ahi',
 'ahora',
 'ahí',
 'al',
 'algo',
 'alguna',
 'algunas',
 'alguno',
 'algunos',
 'algún']

### Como nota el parametro StopWords tiene las palabras por defecto en ingles, sin embargo en español no las tiene, es decir que al vectorizador podriamos pasarle 'english' y sklearn ya tiene las stopwords en ingles definidos

In [7]:
vectorizador.fit_transform(noticias.descripcion)

/home/buitrago/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 

<16495x59952 sparse matrix of type '<class 'numpy.float64'>'
	with 397698 stored elements in Compressed Sparse Row format>

In [8]:
noticias.shape

(16495, 2)

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

from scipy.sparse import issparse

class transformadorSparse(BaseEstimator):
    
    def __init__(self, return_copy=True):
        self.return_copy = return_copy
        self.is_fitted = False
        
    def transform(self, X, y=None):
        if issparse(X):
            return X.toarray()
        elif self.return_copy:
            return X.copy()
        else:
            return X
    
    def fit(self, X, y=None):
        self.is_fitted = True
        return self
    
    def fit_transform(self, X, y=None):
        return self.transform(X=X,y=y)

In [10]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

scikit-learn tiene tres implementaciones del clasificador que hacen esto y todas utilizan distribuciones de probabilidad distintas

## GaussianNB: 
asume que los datos siguen una distribución Gaussiana

In [11]:
pipeline_gaussiano = make_pipeline(
    vectorizador,
    transformadorSparse(),
    GaussianNB()
)

In [12]:
noticias_seccionado = pd.DataFrame()
noticias_seccionado["descripcion"] = noticias.descripcion[:2000]
noticias_seccionado["categoria"] = noticias.categoria[:2000]

In [13]:
pipeline_gaussiano.fit(X=noticias_seccionado.descripcion, y=noticias_seccionado.categoria)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(stop_words=['0', '1', '2', '3', '4', '5', '6',
                                             '7', '8', '9', '_', 'a',
                                             'actualmente', 'acuerdo',
                                             'adelante', 'ademas', 'además',
                                             'adrede', 'afirmó', 'agregó',
                                             'ahi', 'ahora', 'ahí', 'al',
                                             'algo', 'alguna', 'algunas',
                                             'alguno', 'algunos', 'algún', ...],
                                 strip_accents='unicode')),
                ('transformadorsparse', transformadorSparse()),
                ('gaussiannb', GaussianNB())])

# Nota

La matriz es muy grande y ocupa toda la RAM del computador por eso se crea el DataSet *noticias_seccionado*

Cuando tenga un mejor PC ver el video **108 naive bayes classifier**

In [14]:
pipeline_gaussiano.predict(noticias_seccionado.descripcion)

array(['cultura', 'cultura', 'tecnología', ..., 'cultura', 'cultura',
       'cultura'], dtype='<U10')

In [16]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipeline_gaussiano, noticias_seccionado.descripcion, noticias_seccionado.categoria, scoring="f1")

/home/buitrago/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

Al hacer validación cruzada esto arroja un error ya que nuestra variable objetivo es multiclase y la media definida para el cross_val_score es binaria 

In [17]:
from sklearn.metrics import f1_score

In [18]:
f1_score?

en la documentación se puede ver que la puntuación f1 nos permite 5 tipos de calculo para la media
* **binary**: Devuelve la puntuación para la clase especificada en el argumento *pos_label*. Solo se puede aplicar en clasificación binaria. Este es el caso que se encuentra en formas de evaluación para modelos de clasificación

* **micro**: Cuenta el número total de verdaderos positivos (TP), falsos negativos (FN) y falsos positivo (FP) y calcula una precisión y sensibilidad total y obtiene el F1, este metodo es mejor cuando se tienen clases no balanceadas (muchos casos más de una clase que de las demas o una distribución de datos asimetricas)

* **macro**: calcula la precisión y la sensibilidad media de cada clase, hace su media aritmetica y calcula el parametro F1. (no tiene en cuenta las clases no balanceadas)

* **weighted**: Calcula la precisión y sensibilidad media de cada clase, hace su media ponderada por el número de observaciones de cada clase y calcula el parametro F1.

* **samples**

Para solucionar este problema le pasamos el parametro *average* y le pasamos un tipo de calculo para multietiqueta

In [21]:
def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return f1_score(y, preds, average="micro")

cross_val_score(pipeline_gaussiano, noticias_seccionado.descripcion, noticias_seccionado.categoria, 
                scoring=f1_multietiqueta)

/home/buitrago/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 

array([0.65  , 0.65  , 0.62  , 0.6475, 0.64  ])

Se puede restringir la cantidad de palabras que considera el modelo con el parametro *max_features*

In [22]:
pipeline_gaussiano = make_pipeline(
    TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es, max_features=500),
    transformadorSparse(),
    GaussianNB()
)

In [23]:
cross_val_score(pipeline_gaussiano, noticias_seccionado.descripcion, noticias_seccionado.categoria, 
                scoring=f1_multietiqueta)

/home/buitrago/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 

array([0.49  , 0.45  , 0.4875, 0.445 , 0.445 ])

Las implementaciones más utilizadas para clasificación de texto son la [MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) y la [BernoulliNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html)

In [24]:
pipeline_multinomial = make_pipeline(
    TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es, max_features=1000),
    transformadorSparse(),
    MultinomialNB()
)

In [25]:
cross_val_score(pipeline_multinomial, noticias_seccionado.descripcion, noticias_seccionado.categoria, 
                scoring=f1_multietiqueta)

/home/buitrago/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 

array([0.6675, 0.6775, 0.6725, 0.66  , 0.67  ])

Multinomial efectivamente funciona mejor que el Gaussiano

Para Bernoulli debemos usar [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) ya que esta distribución solo considera dos valores 0 o 1 por tanto necesitamos las palabras como vectores binarios, además del parametro *binary=True* para que devuelva 1 o 0 en vez del numero de veces que aparece la palabra

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizador_count = CountVectorizer(stop_words=stopwords_es, binary=True, strip_accents="unicode", 
                                     max_features=1000)

El *CountVectorizer* permite ver cuantas veces aparece cada palabra en el corpus

In [27]:
vectorizador_count.fit(noticias_seccionado.descripcion, noticias_seccionado.categoria)
vectorizador_count.vocabulary_

/home/buitrago/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 

{'tierra': 925,
 '30': 25,
 'anos': 77,
 'epoca': 337,
 'principal': 763,
 'estudio': 365,
 'internacional': 504,
 'publicado': 795,
 'nature': 641,
 'pasar': 705,
 'proyecto': 789,
 'union': 947,
 'europea': 368,
 'llegaron': 549,
 'metodo': 598,
 'aprender': 88,
 'lengua': 530,
 'espanol': 348,
 'sistema': 880,
 'espanoles': 351,
 'puedan': 800,
 'entender': 332,
 'china': 167,
 'confirmado': 209,
 'exito': 375,
 'operacion': 678,
 'forma': 407,
 'totalmente': 932,
 'robot': 846,
 'logro': 556,
 'personal': 722,
 'humano': 471,
 'carrera': 155,
 'actor': 49,
 'director': 287,
 'escritor': 343,
 'decenas': 257,
 'peliculas': 708,
 'figuras': 400,
 'importantes': 484,
 'historia': 461,
 'cine': 176,
 'humor': 473,
 'mayoria': 586,
 'encuentran': 325,
 'publico': 797,
 'online': 677,
 'mejores': 590,
 'biblioteca': 121,
 'museo': 631,
 'finalmente': 402,
 'siglo': 876,
 'musica': 632,
 'texto': 922,
 'creado': 234,
 'nombre': 653,
 'obra': 664,
 'video': 975,
 'hombre': 464,
 'comida': 

In [28]:
pipeline_bernoulli = make_pipeline(
    vectorizador_count,
    transformadorSparse(),
    BernoulliNB()
)

In [29]:
cross_val_score(pipeline_bernoulli, noticias_seccionado.descripcion, noticias_seccionado.categoria, 
                scoring=f1_multietiqueta)

/home/buitrago/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 

array([0.6725, 0.6475, 0.64  , 0.6   , 0.6525])

Este modelo funciona incluso mejor que los dos anteriores